In [3]:
import os

# gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# nltk
from nltk.corpus import stopwords

# spacy
import spacy          
#nlp = spacy.load("da_core_news_lg")

#visualisering
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()


### Stopord fra NLTK

In [91]:
# stopw = stopwords.words("danish")

In [93]:
# len(stopw)

94

### Stopord fra liste

In [4]:
stopord = open("stopord.txt", encoding = "utf8").read().split("\n")

In [5]:
print(stopord)

['ad', 'af', 'aldrig', 'alene', 'alle', 'allerede', 'alligevel', 'alt', 'altid', 'anden', 'andet', 'andre', 'at', 'bag', 'bare', 'begge', 'bl.a.', 'blandt', 'blev', 'blive', 'bliver', 'burde', 'bør', 'ca.', 'da', 'de', 'dem', 'den', 'denne', 'dens', 'der', 'derefter', 'deres', 'derfor', 'derfra', 'deri', 'dermed', 'derpå', 'derved', 'det', 'dette', 'dig', 'din', 'dine', 'disse', 'dit', 'dog', 'du', 'efter', 'egen', 'ej', 'eller', 'ellers', 'en', 'end', 'endnu', 'ene', 'eneste', 'enhver', 'ens', 'enten', 'er', 'et', 'f.eks.', 'far', 'fem', 'fik', 'fire', 'flere', 'flest', 'fleste', 'for', 'foran', 'fordi', 'forrige', 'fra', 'fx', 'få', 'får', 'før', 'først', 'gennem', 'gjorde', 'gjort', 'god', 'godt', 'gør', 'gøre', 'gørende', 'ham', 'han', 'hans', 'har', 'havde', 'have', 'hej', 'hel', 'heller', 'helt', 'hen', 'hende', 'hendes', 'henover', 'her', 'herefter', 'heri', 'hermed', 'herpå', 'hos', 'hun', 'hvad', 'hvem', 'hver', 'hvilke', 'hvilken', 'hvilkes', 'hvis', 'hvor', 'hvordan', 'hvore

Tilføjelse af ord til stopordslisten

In [71]:
# stopord = stopord + ["aaa", "bbb"]

### Indlæs tekster 

In [6]:
taler_0 = [] # opretter tom liste

path = os.path.join("Statsministerens_nytårstaler") # bruges for at undgå mac/pc-problemerne med absolutte stinavne

for fil in os.scandir(path): # for-loop
    with open (fil, encoding = "utf8") as f: # context manager
        taler_0.append(f.read().replace("\n"," ").replace("*"," ").replace("."," ").replace(",", " ").lower()) # tilføj renset tekst til liste

In [7]:
taler_0[0][0:50]

'det nye år fremstilles ofte som et nyfødt barn  de'

### Lemmatization

In [8]:
def lemmatization(taler, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("da_core_news_lg", disable=["parser","ner"])
    taler_ud = []
    for t in taler:
        doc = nlp(t)
        ny_tale = []
        for token in doc:
            if token.pos_ in allowed_postags and token.text not in stopord:
                ny_tale.append(token.lemma_)
        taler_ud.append(ny_tale)
    return(taler_ud)
    
                
    

In [9]:
lemma_taler = lemmatization(taler_0)

In [24]:
#print(lemma_taler)

In [20]:
#for t in lemma_taler:
#    print(len(t))

In [21]:
#for t in taler_0:
#    print(len(t))

In [22]:
#for t in taler_ord:
#   print(len(t))

### Corpus og dictionary

In [25]:
id2word = corpora.Dictionary(lemma_taler)

corpus = []
for t in lemma_taler:
    new = id2word.doc2bow(t)
    corpus.append(new)

In [33]:
print(corpus[0][0:20])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 4), (13, 1), (14, 3), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1)]


In [31]:
word = id2word[[0][:1][0]]
print(word)

administrativ


### LDA model

In [39]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                            id2word = id2word,
                                            num_topics = 15,
                                            random_state = 100,
                                            update_every = 1,
                                            chunksize = 100,
                                            passes = 10,
                                            alpha = "auto")

### Visualisering af data

In [42]:
pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
#vis

lda_viz = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)

In [43]:
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.090052 -0.053365       1        1  43.543322
4      0.108139 -0.152512       2        1  13.032099
14     0.044267 -0.149846       3        1  11.517806
12     0.138790  0.114511       4        1   7.967832
1      0.152538 -0.092001       5        1   7.350864
9      0.169979 -0.003674       6        1   5.406749
2      0.125387  0.059381       7        1   4.852629
0     -0.047458 -0.150996       8        1   3.576494
3     -0.130470 -0.076277       9        1   1.760473
11    -0.044273  0.151345      10        1   0.973336
5     -0.116321  0.067629      11        1   0.003708
10    -0.119964  0.069923      12        1   0.003676
8     -0.122694  0.071344      13        1   0.003674
13    -0.123590  0.072071      14        1   0.003672
7     -0.124383  0.072467      15        1   0.003665, topic_info=      Term        Freq       Total Category  logprob  loglift
18    barn  141.000000  141.000000  Default  30.0000  30.0000
468     år  620.000000  620.000000  Default  29.0000  29.0000
48   dansk  295.000000  295.000000  Default  28.0000  28.0000
228   land  343.000000  343.000000  Default  27.0000  27.0000
152    god  275.000000  275.000000  Default  26.0000  26.0000
..     ...         ...         ...      ...      ...      ...
243    liv    0.000315   98.936059  Topic15  -8.4172  -2.4427
706    tro    0.000323  145.846319  Topic15  -8.3917  -2.8052
390  sætte    0.000320  138.835667  Topic15  -8.4010  -2.7653
145   give    0.000322  160.754511  Topic15  -8.3972  -2.9081
168   hele    0.000315  126.475718  Topic15  -8.4165  -2.6876

[1180 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1819      1  0.893407  1970’erne
4727      1  0.575539  1990’erne
4727     10  0.575539  1990’erne
2612      1  0.974180    30’erne
1432      1  0.392822   aflevere
...     ...       ...        ...
980       5  0.144930          –
980       7  0.108698          –
980       8  0.018116          –
980       9  0.018116          –
4726      2  0.968935          •

[1926 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 15, 13, 2, 10, 3, 1, 4, 12, 6, 11, 9, 14, 8])

In [18]:
taler_ord = []

for t in taler_0:                  
    taler_ord.append(t.split())

In [77]:
for t in taler_ord:
    print(len(t))

1947
2053
1249
1776
1891
1172
1229
0
1206
1551
1991
0
1491
1930
0
1807
1810
2031
1766
1578
1696
1525
1563
0
1683
1777
1829
1704
1922
1806
1768
1894
1716
1940
1893
1727
2023
1961
2011
2182
1969
1483
1387
1661
961
1611
1600
1606
1619
2113
2091
2102
2183
1885
2047
1953
2111
1871
2189
1891
2289
1894


In [78]:
for t in taler_ord:
    for w in t:
        if w in stop:
            t.remove(w)  

In [79]:
for t in taler_ord:
    print(len(t))

1254
1302
856
1153
1234
770
839
0
770
1024
1309
0
983
1224
0
1177
1172
1323
1143
1020
1091
966
985
0
1083
1164
1200
1136
1223
1177
1128
1246
1151
1244
1252
1143
1327
1264
1301
1409
1292
989
912
1106
631
1083
1056
1074
1096
1391
1390
1365
1433
1232
1336
1275
1401
1236
1470
1276
1521
1266


In [80]:
taler_ord[0]

['Det',
 'nye',
 'år',
 'fremstilles',
 'ofte',
 'nyfødt',
 'barn',
 'nysgerrigt',
 'kigger',
 'nye',
 'verden',
 'gamle',
 'år',
 'affældig',
 'olding',
 'liste',
 'baggrunden',
 'ynder',
 'foregive',
 'begynde',
 'frisk',
 'grunden',
 '–',
 'begynde',
 'forfra',
 'nye',
 'år',
 'Det',
 'Vi',
 'tager',
 'problemerne',
 '–',
 'uløste',
 '–',
 'erfaringen',
 'lytte',
 'Vi',
 'sidste',
 'dage',
 'gamle',
 'år',
 'gå',
 'havet',
 'karske',
 'vejr',
 'prøve',
 'lade',
 'krop',
 'sind',
 'blæse',
 'ren',
 'gamle',
 'slagger',
 '–',
 'alligevel',
 'aftenhvilen',
 'melder',
 'nået',
 'passere',
 'revy',
 'De',
 'gode',
 'forsætters',
 'time',
 'inde',
 'Og',
 'begynde',
 'nye',
 'år',
 'tak',
 'alt',
 'udført',
 '1960',
 'Tak',
 'glemt',
 'årets',
 'løb',
 'al',
 'dets',
 'travlhed',
 'På',
 'regeringens',
 'vegne',
 'ønsker',
 'lyst',
 'fredeligt',
 'nytår',
 'vort',
 'land',
 'danske',
 'Vore',
 'varme',
 'tanker',
 'gode',
 'ønsker',
 'går',
 'årets',
 'første',
 'dag',
 'Færøerne',
 'Grøn